# HW#3

#### So far, we have collected CIKs for each of the Mutual Funds, then looked up the links of all the 13F HRs and the Information Tables, and identified them either text tables or xml tables. In class, we  obtained the information from the xml tables.  In this assignment we will obtain information from the text file.  These are not as nicely structured as the xml output.   In a csv (HW_Mutual_Fund_Info_Table_Link.csv) you will find a few of these links.  Your goal for this part of the homework is to obtain the link to the text files from the attached file  (HW_Mutual_Fund_Info_Table_Link.csv).  Then you will write a code that will go to the links of all the linked text files, and extract some columns.  Do not use Beutiful Soup for this assignment.   We provide some initial code to guide your initial steps:

#### First we have to collect the links of the text Info Tables in lists:

In [60]:
import urllib
xml_link = []
xml_Name = []
xml_date = []
text_link = []
text_Name = []
text_date = []

input_file = open('HW_Mutual_Fund_Info_Table_Link.csv', 'r')

rows = input_file.readlines()
input_file.close()

# Your Code on Enumerating the Lists. 

for row in range(len(rows)):
    new = rows[row].split(",")
    if new[4] == "xml":
        xml_link.append(new[3])
        xml_Name.append(new[1])
        xml_date.append(new[2])
    elif new[4] == 'text':
        text_link.append(new[3])
        text_Name.append(new[1])
        text_date.append(new[2])

### Part 1 Beautiful Soup

#### Keep only the links that correspond to a date after 2010 (Don't inlude 2010, start at 2011).  Hint: you can use the datetime library.

In [61]:
from datetime import datetime

#Use the following list to store the filtered values.
filtered_dates = []
filtered_name = []
filtered_link = []

#Enter code here to to keep only the dates corresponding to after 2010.  
for row in range(len(xml_date)):
    if datetime.strptime(xml_date[row],'%Y-%m-%d') >= datetime(2011,1,1):
        filtered_dates.append(xml_date[row])
        filtered_name.append(xml_Name[row]) 
        filtered_link.append(xml_link[row])

#### Next, for each text link, extract the name of issuer, CUSIP, and the Quantity of shares.  You will also want to keep track of the mutual fund name as well as the filing report date.  

#### Your output file should have 5 columns.  The first is the issue date of the form which can be found in the filtered_date list (this will be repeated for the same form).  The second is the mutual fund name which can be found in the filtered_name list (this will be repeated).  The third, fourth and fifith are the name of issuer, CUSIP, and shares respectively.  Make sure to account for the fact that while some of the text files have the same formatting, others do not.  This means you will have to look through them to make sure your code works for the each text file.

In [62]:
import urllib.request
import bs4 as bs
import codecs

issue_date = []
mutual_fund_name = [] 
name_of_issuer = [] 
CUSIP = [] # CUSIP number
shares = [] # No. of Shares of the company in the Mutual Fund

for link in range(len(filtered_link)):  
    url = 'https://www.sec.gov'+filtered_link[link]
    html = urllib.request.urlopen(url).read().decode('utf-8')

    soup = bs.BeautifulSoup(html,"html.parser")
    
    infotable = soup.find_all("table", summary = "Form 13F-NT Header Information")
    
    for row in infotable[0].find_all('tr')[3:]:
        columns = row.find_all('td')
        name_of_issuer.append(columns[0].getText())
        CUSIP.append(columns[2].getText())
        shares.append(columns[4].getText())
        mutual_fund_name.append(filtered_name[link])
        issue_date.append(filtered_dates[link])
len(name_of_issuer)

2830

### Part 2: Pandas

In [63]:
from datetime import datetime

#Use the following list to store the filtered values.
new_dates = []
new_name = []
new_link = []

#Enter code here to to keep only the dates corresponding to after 2010.  
for row in range(len(text_date)):
    if datetime.strptime(text_date[row],'%Y-%m-%d') >= datetime(2011,1,1):
        new_dates.append(text_date[row])
        new_name.append(text_Name[row]) 
        new_link.append(text_link[row]) 

In [64]:
for link in range(len(new_link)):
    url = 'https://www.sec.gov'+new_link[link]
    html = urllib.request.urlopen(url).read().decode('utf-8')
    
    while html.find("<CAPTION>") != -1:
        index = html.find("<CAPTION>") 
        html2 = html[index+10:]
        index_Start = html2.find("<")
        index_End = html2.find("</TABLE>")
        
        if html2.find("GRAND")!= -1:
            index_End = html2.find("GRAND")
            html2 = html2[index_Start:index_End]
        else:
            html2 = html2[index_Start:index_End]  #going closer to the results table
   
        findIndex = html2.split("\n")
        #Find Issuer Name Index for each line
        NameIndex_Start = findIndex[0].find("<")
        NameIndex_End = findIndex[0][NameIndex_Start+1:].find("<")+1
        #Find CUSIP Index for each line
        CUSIP_Start = findIndex[0].find("<",NameIndex_End+1)
        CUSIP_End = findIndex[0].find("<",CUSIP_Start+1)
        #Fine Share Index for each line
        Share_Start = findIndex[0].find("<",CUSIP_End+1)
        Share_End = findIndex[0].find("<",Share_Start+1)
    
        # for file1 and file7, we have different startlines to get data.
        if link == 0:
            startline = 3
        elif link == 6:
            startline = 4
        else:
            startline = 1
    
        for row in range(startline,len(findIndex)):
            # remove blank items, if the issuer name is seperated in 2 lines, add the second line to the previous line.
            if findIndex[row][NameIndex_Start:NameIndex_End].strip() != '':
                if findIndex[row][CUSIP_Start:CUSIP_End]!='':
                    issue_date.append(new_dates[link])
                    mutual_fund_name.append(new_name[link])
                    name_of_issuer.append(findIndex[row][NameIndex_Start:NameIndex_End].strip())
                    CUSIP.append(findIndex[row][CUSIP_Start:CUSIP_End].strip())
                    shares.append(findIndex[row][Share_Start:Share_End].strip())
                else:
                    name_of_issuer[-1] = name_of_issuer[-1]+findIndex[row][NameIndex_Start:NameIndex_End].strip()
        html = html[index_End:]


#### Finally, you write down the lists in the following file:

In [65]:
# Your code goes here
import pandas as pd

result = pd.DataFrame({"Issue_date":issue_date,"Name":mutual_fund_name,"Name_of_Issuer":name_of_issuer,"CUSIP":CUSIP,"Shares":shares})

result.to_csv("Mutual_Fund_Info_Table_pandas.csv")
